In [1]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz, process
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/ingredients_df.csv')

pd.set_option('max_columns', None)

recipe_df.head()

,recipe_name,recipe_url,ingredients_list,ingredient,variable,num_ingredients,category,main_category,tipo_plato,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free
0,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",patatas pequeñas rojas,0,12,ensaladas,principal,['Primeros platos y entrantes'],60.0,252.0,0,0,0,0,0
1,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",foie micuit,0,9,ensaladas,principal,['Primeros platos y entrantes'],25.0,386.0,0,0,0,0,0
2,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",bacalao desalado,0,16,ensaladas,principal,['Primeros platos y entrantes'],35.0,NaN,0,0,0,0,0
3,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",langostinos,0,12,ensaladas,principal,['Primeros platos y entrantes'],25.0,176.0,0,0,0,0,0
4,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",pechugas de pollo,0,16,ensaladas,principal,['Primeros platos y entrantes'],40.0,203.0,0,0,0,0,0


In [3]:
input_list = ['huevos', 'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche', 'mantequilla',
              'harina', 'sal', 'azúcar', 'aceite', 'zanahorias', 'queso', 'bacon', 'lechuga', 'calabacín',
              'patatas', 'pollo', 'atún', 'garbanzos', 'judías verdes', 'jamón', 'jamón cocido', 'pan', 
              'pasta']

In [4]:
def fuzzy_columns(column, input_string):
    ratio = fuzz.WRatio(column, input_string)
    return ratio

In [5]:
for ingred in input_list:
    recipe_df[ingred] = recipe_df.apply(lambda x: fuzzy_columns(x["ingredient"], ingred), axis=1)

In [6]:
recipe_df

,recipe_name,recipe_url,ingredients_list,ingredient,variable,num_ingredients,category,main_category,tipo_plato,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,lechuga,calabacín,patatas,pollo,atún,garbanzos,judías verdes,jamón,jamón cocido,pan,pasta
0,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",patatas pequeñas rojas,0,12,ensaladas,principal,['Primeros platos y entrantes'],60.0,252.0,0,0,0,0,0,45,32,34,22,36,36,40,30,30,36,30,36,72,18,39,34,90,18,60,40,45,45,24,60,72
1,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",foie micuit,0,9,ensaladas,principal,['Primeros platos y entrantes'],25.0,386.0,0,0,0,0,0,15,28,21,42,12,25,27,15,0,18,45,19,18,18,39,11,13,18,30,10,9,22,34,0,18
2,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",bacalao desalado,0,16,ensaladas,principal,['Primeros platos y entrantes'],35.0,NaN,0,0,0,0,0,30,44,34,22,36,19,30,30,90,54,30,36,36,54,26,45,39,36,30,40,36,22,37,30,36
3,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",langostinos,0,12,ensaladas,principal,['Primeros platos y entrantes'],25.0,176.0,0,0,0,0,0,30,22,42,32,36,18,36,35,30,18,30,48,36,38,26,32,33,25,60,40,26,45,45,60,38
4,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",pechugas de pollo,0,16,ensaladas,principal,['Primeros platos y entrantes'],40.0,203.0,0,0,0,0,0,45,91,34,24,34,54,32,30,30,18,30,26,36,18,77,24,26,90,30,30,41,22,32,30,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57309,Pastel de tres colores y tres sabores,https://www.lecturas.com/recetas/pastel-tres-c...,"['mantequilla', 'azúcar', 'huevo', 'aroma de v...",azúcar,30,33,postres y dulces,postre,['Postres'],240.0,NaN,0,0,0,0,0,0,19,18,0,40,20,25,36,30,100,36,40,0,40,33,36,33,0,30,36,24,22,36,30,40
57310,Tarta de cubos de chocolate con toffee,https://www.lecturas.com/recetas/tarta-cubos-c...,"['mantequilla', 'azúcar', 'huevo', 'harina', '...",chocolate negro fondant,31,33,postres y dulces,postre,['Postres'],360.0,NaN,0,0,0,0,0,30,36,45,34,36,36,31,34,30,36,45,27,36,36,27,45,27,36,60,30,36,45,38,60,36
57311,Pastel de tres colores y tres sabores,https://www.lecturas.com/recetas/pastel-tres-c...,"['mantequilla', 'azúcar', 'huevo', 'aroma de v...",chocolate negro fondant,31,33,postres y dulces,postre,['Postres'],240.0,NaN,0,0,0,0,0,30,36,45,34,36,36,31,34,30,36,45,27,36,36,27,45,27,36,60,30,36,45,38,60,36
57312,Tarta de cubos de chocolate con toffee,https://www.lecturas.com/recetas/tarta-cubos-c...,"['mantequilla', 'azúcar', 'huevo', 'harina', '...",cacao en polvo puro,32,33,postres y dulces,postre,['Postres'],360.0,NaN,0,0,0,0,0,30,51,22,22,36,36,24,30,30,54,45,28,36,54,26,45,26,72,30,30,26,22,38,30,18


In [7]:
input_list

['huevos',
 'pechiga de pollo',
 'lentejas',
 'pimiento',
 'arroz',
 'leche',
 'mantequilla',
 'harina',
 'sal',
 'azúcar',
 'aceite',
 'zanahorias',
 'queso',
 'bacon',
 'lechuga',
 'calabacín',
 'patatas',
 'pollo',
 'atún',
 'garbanzos',
 'judías verdes',
 'jamón',
 'jamón cocido',
 'pan',
 'pasta']

In [8]:
filter_huevos = recipe_df['huevos'] > 80

recipe_df[filter_huevos]["ingredient"].value_counts()

huevo                     1373
huevos                     723
yema de huevo              329
clara de huevo             194
yemas de huevo             101
claras de huevo             76
huevos de codorniz          35
huevos grandes              10
huevos medianos              6
huevos duros                 2
huevos cocidos               1
fideos chinos de huevo       1
fideos de huevo              1
huevos cocido                1
huevos de chocolate          1
huevos pequeños              1
Name: ingredient, dtype: int64

In [9]:
filter_pechiga_de_pollo = recipe_df['pechiga de pollo'] > 86
recipe_df[filter_pechiga_de_pollo]["ingredient"].value_counts()

pechugas de pollo    86
pechuga de pollo     14
pollo                 4
Name: ingredient, dtype: int64

In [10]:
filter_lentejas = recipe_df['lentejas'] > 86
recipe_df[filter_lentejas]["ingredient"].value_counts()

lentejas                      23
lentejas cocidas              11
lentejas pardinas              8
lentejas pardinas cocidas      8
lentejas rojas                 2
lentejas rojas lavados         1
hummus de lentejas rojas       1
lentejas del tipo pardinas     1
paté de lentejas               1
Name: ingredient, dtype: int64

In [11]:
filter_pimiento = recipe_df['pimiento'] > 86
recipe_df[filter_pimiento]["ingredient"].value_counts()

pimienta                                    1484
pimiento rojo                                318
pimiento verde                               278
pimientos rojos                               89
pimientos verdes                              60
                                            ... 
pimientos morrones asados o del piquillo       1
pimientos choriceros (la pulpa)                1
pimiento choricero de su carne                 1
pimientos choriceros la pulpa                  1
pimientos choriceros de la pulpa               1
Name: ingredient, Length: 83, dtype: int64

In [12]:
filter_arroz = recipe_df['arroz'] > 86
recipe_df[filter_arroz]["ingredient"].value_counts()

arroz                                      148
arroz redondo                               30
arroz integral                              29
arroz basmati                               29
arroz largo                                 29
arroz bomba                                 19
harina de arroz                             18
vinagre de arroz                            16
leche de arroz                              10
arroz salvaje                                9
arroz arborio                                8
arroz carnaroli                              4
arroz inflado de chocolate                   4
fideos de arroz                              4
sirope de arroz                              4
morcilla de arroz                            3
arroz cocido                                 3
oblea de arroz                               3
arroz nerone (venere)                        3
pasta de arroz                               2
arroz jazmín                                 2
arroz inflado

In [13]:
filter_leche = recipe_df['leche'] > 86
recipe_df[filter_leche]["ingredient"].value_counts()

leche                                       987
leche condensada                             43
chocolate con leche                          38
leche desnatada                              37
leche de coco                                30
leche de soja                                26
leche evaporada                              25
chocolate con leche fondant                  24
leche de almendras                           12
dulce de leche                               11
leche de arroz                               10
leche vegetal                                 9
leche semidesnatada                           7
leche de avena                                7
leche tibia                                   4
leche condensada 1 bote pequeño               3
chocolate con leche de cobertura              2
virutas de chocolate con leche                2
leche entera                                  2
leche galletas de vainilla                    1
leche en polvo                          

In [14]:
filter_mantequilla = recipe_df['mantequilla'] > 86
recipe_df[filter_mantequilla]["ingredient"].value_counts()

mantequilla                                    1796
mantequilla una nuez                              9
mantequilla ablandada                             5
mantequilla fundida                               5
mantequilla de cacahuete                          4
mantequilla reblandecida                          4
mantequilla nuez                                  4
mantequilla salada                                3
mantequilla con sal                               3
mantequilla a temperatura ambiente                2
mantequilla 1 nuez                                1
mantequilla light                                 1
mantequilla o margarina                           1
galletas danesas de mantequilla                   1
mantequilla par untar el molde                    1
galletas de mantequilla                           1
mantequilla fría                                  1
mantequilla pomada                                1
mantequilla en un trozo                           1
mantequilla 

In [15]:
filter_harina = recipe_df['harina'] > 86
recipe_df[filter_harina]["ingredient"].value_counts()

harina                                    1674
harina de maíz                             270
harina de trigo                             81
harina de fuerza                            61
harina integral                             30
harina de maíz maicena                      22
harina de maíz (maicena)                    18
harina de almendra                          18
harina de arroz                             18
harina de garbanzos                         10
harina de repostería                         5
harina de fuerza + 4 cucharadas              5
harina de trigo sarraceno                    5
harina de repostería sin gluten              5
harina de tempura                            4
harina de avena                              4
harina de trigo integral                     3
harina de maíz de la amarilla                2
harina de coco                               2
harina de espelta integral                   2
harina leudante                              2
harina de maí

In [16]:
filter_sal = recipe_df['sal'] > 86
recipe_df[filter_sal]["ingredient"].value_counts()

sal                      3667
salsa mayonesa            102
salsa de soja              93
salsa de tomate            72
salmón ahumado             52
                         ... 
alga nori en sal            1
salsa mayonesa ligera       1
tomate de ensalada          1
salchichas tipo viena       1
sal ahumada                 1
Name: ingredient, Length: 121, dtype: int64

In [17]:
filter_azúcar = recipe_df['azúcar'] > 86
recipe_df[filter_azúcar]["ingredient"].value_counts()

azúcar                                2115
azúcar glas                            861
azúcar moreno                          258
azúcar avainillado                      85
azúcar de coco                          16
                                      ... 
azúcar avainillado pequeño               1
azúcar glas sin gluten                   1
bolitas de azúcar de colores rojas       1
azúcar glas galletas de vainilla         1
perla de azúcar plateadas y rosas        1
Name: ingredient, Length: 67, dtype: int64

In [18]:
filter_aceite = recipe_df['aceite'] > 86
recipe_df[filter_aceite]["ingredient"].value_counts()

aceite de oliva                                     2440
aceite                                               578
aceite de girasol                                    121
aceite de oliva virgen extra                          99
aceite de oliva suave                                 74
                                                    ... 
aceitunas rellenas de anchoa filetes en aceite         1
ventresca de atún en conserva en aceite                1
habas pequeñas en aceite                               1
filetes de caballa en aceite                           1
ventresca de atún en conserva en aceite de oliva       1
Name: ingredient, Length: 61, dtype: int64

In [19]:
filter_zanahorias = recipe_df['zanahorias'] > 86
recipe_df[filter_zanahorias]["ingredient"].value_counts()

zanahoria                                                                           407
zanahorias                                                                          263
zanahorias baby                                                                       8
zanahorias rallada                                                                    4
zanahorias pequeñas                                                                   3
conservas de verduras guisantes y zanahorias                                          1
zanahorias mediana                                                                    1
fondant en forma de zanahorias                                                        1
zanahorias peladas y ralladas                                                         1
zanahorias con las hojas                                                              1
calabacín sirve cualquier resto de verduras cocidas (acelgas puerros zanahorias)      1
zanahorias medianas             

In [20]:
filter_queso = recipe_df['queso'] > 86
recipe_df[filter_queso]["ingredient"].value_counts()

queso blanco                   245
queso rallado                  135
queso parmesano rallado        105
queso mascarpone                75
queso parmesano                 64
                              ... 
queso burrata o mozzarella       1
queso parmesano en virutas       1
queso de cabra rulo              1
queso de cabra sin corteza       1
queso mozzarella en lonchas      1
Name: ingredient, Length: 130, dtype: int64

In [21]:
filter_bacon = recipe_df['bacon'] > 86
recipe_df[filter_bacon]["ingredient"].value_counts()

Series([], Name: ingredient, dtype: int64)

In [22]:
filter_lechuga = recipe_df['lechuga'] > 86
recipe_df[filter_lechuga]["ingredient"].value_counts()

lechuga                                                 54
lechugas                                                33
cogollo de lechuga                                      18
lechuga unas hojas                                      15
lechuga variadas                                         9
lechuga brotes variados                                  9
lechugas variada                                         8
lechugas brotes variados                                 8
lechuga brotes                                           7
cogollos de lechuga                                      6
lechuga hoja de roble                                    6
lechugas brotes                                          5
lechuga unas hojas variadas                              5
lechuga hojas variadas                                   5
lechuga romana                                           4
lechuga unos brotes                                      4
lechugas romana                                         

In [23]:
filter_calabacín = recipe_df['calabacín'] > 80
recipe_df[filter_calabacín]["ingredient"].value_counts()

calabacín             261
calabacines            58
calabacín pequeño      12
calabacín grande        4
calabacín mediano       3
calabacín medianos      3
calabacín pequeños      3
calabacín troceado      1
calabacín rallado       1
calabacín baby          1
flor de calabacín       1
calabacín bola          1
Name: ingredient, dtype: int64

In [24]:
filter_patatas = recipe_df['patatas'] > 86
recipe_df[filter_patatas]["ingredient"].value_counts()

patatas                               211
patata                                200
patatas medianas                       53
patatas pequeñas                       24
patatas grandes                         6
patatas baby                            6
paté                                    5
patatas nuevas pequeñas                 2
patatas pequeñas o 1 grande             1
patatas pequeñas rojas                  1
patatas violeta                         1
patatas tipo baby                       1
patatas medianas o 4 pequeñas           1
patatas cocidas                         1
patatas del tipo violeta                1
patatas baby cocidas                    1
patatas medianas y alargadas            1
patatas nuevas                          1
patatas pequeñas cocidas envasadas      1
patatas chips                           1
Name: ingredient, dtype: int64

In [25]:
filter_pollo = recipe_df['pollo'] > 86
recipe_df[filter_pollo]["ingredient"].value_counts()

pechugas de pollo                          86
caldo de pollo                             76
muslos de pollo                            18
pechuga de pollo                           14
caldo de pollo o de ave                    12
                                           ..
pechugas de pollo deshuesadas y limpias     1
caldo de verduras o de pollo o carne        1
pollo de unos 1,5 kg cortado en octavos     1
muslos de pollo deshuesados y troceados     1
pollo abierto y deshuesado                  1
Name: ingredient, Length: 78, dtype: int64

In [26]:
filter_atun = recipe_df['atún'] > 86
recipe_df[filter_atun]["ingredient"].value_counts()

atún en conserva          31
atún                      11
lomos de atún              5
atún en escabeche          4
filetes de atún            2
ventresca de atún          2
bonito o atún              1
lomos de atún fresco       1
atún o bonito              1
atún en filetes            1
atún fresco y limpio       1
atún de 150 g cada uno     1
Name: ingredient, dtype: int64

In [27]:
filter_garbanzos = recipe_df['garbanzos'] > 86
recipe_df[filter_garbanzos]["ingredient"].value_counts()

garbanzos cocidos                   47
garbanzos                           21
harina de garbanzos                 10
garbanzos remojados                  3
garbanzos secos                      2
tempeh de garbanzos                  2
garbanzos cocidos en conserva        1
garbanzos ya remojados               1
agua de cocción de los garbanzos     1
garbanzos remojados 24 horas         1
garbanzos en remojo                  1
Name: ingredient, dtype: int64

In [28]:
filter_judías_verdes = recipe_df['judías verdes'] > 86
recipe_df[filter_judías_verdes]["ingredient"].value_counts()

judías verdes                   71
judía verde                     60
judías verdes finas              6
judías verdes redondas           4
judías verdes redonda            3
judías verdes planas cocidas     3
judías verdes planas             2
judías verdes redondas finas     1
Name: ingredient, dtype: int64

In [29]:
filter_jamón = recipe_df['jamón'] > 86
recipe_df[filter_jamón]["ingredient"].value_counts()

jamón serrano                        91
jamón cocido                         54
hueso de jamón                       16
jamón serrano en lonchas             16
jamón serrano en daditos             11
jamón serrano en taquitos             9
jamón cocido en lonchas               8
jamón serrano finas                   6
jamón serrano en dados                5
jamón serrano en lonchas finas        4
jamón de pato                         4
jamón serrano en tacos                4
jamón cocido en un trozo              4
jamón cocido en dados                 3
jamón ibérico en virutas              3
jamón cocido en una loncha            3
jamón ibérico                         3
jamón fresco de cerdo                 3
jamón serrano a taquitos              3
caldo de jamón                        2
jamón de pato en lonchas              2
jamón serrano a tacos                 2
jamón serrano o curado                2
jamón serrano lonchas finas           2
jamón de pavo                         2


In [30]:
filter_pan = recipe_df['pan'] > 86
recipe_df[filter_pan]["ingredient"].value_counts()

pan rallado                 165
pan                          82
pan de molde                 38
pan de molde sin corteza     28
miga de pan                  23
                           ... 
panecillo la miga             1
minipanecillos                1
pan de focaccia               1
pan con cereales              1
pan de pipas                  1
Name: ingredient, Length: 85, dtype: int64

In [34]:
filter_pasta = recipe_df['pasta'] > 86
recipe_df[filter_pasta]["ingredient"].value_counts()

pasta brisa                         50
pasta brick                         30
pasta filo                          27
pasta brisa refrigerada             13
espirales de pasta                  12
                                    ..
sandías en pasta                     1
pasta de frambuesa o esencia         1
galets de pasta del nº 3 al 5        1
pasta brisa un disco refrigerado     1
galets de pasta o caracolas          1
Name: ingredient, Length: 68, dtype: int64